In [ ]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt


from copy import deepcopy
import os
import dotenv

from mp_api.client import MPRester
from pymatgen.io.ase import AseAtomsAdaptor


In [ ]:
## Global Config ##

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")
MP_API_KEY = os.environ.get("MP_API_KEY")

MP_ID = "mp-1211504"

In [ ]:
## Get the structure from the Materials Project and convert to ASE

def get_cif(material_id):
    m = MPRester(MP_API_KEY)
    structure = m.get_structure_by_material_id(material_id)
    return structure

cif = get_cif(MP_ID)  
reduced_formula = str(cif).split("\n")[1].split(": ")[1]

AAA = AseAtomsAdaptor()
lattice = AAA.get_atoms(cif)

outdir = "./{}".format(reduced_formula)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
## Quantum Espresso Config ## 
pseudopotentials = {}
for symbol in np.unique(lattice.get_chemical_symbols()):
    for pp in os.listdir(PPDIR):
        if pp.lower().startswith(symbol.lower()) and not (pp.lower()[1].isalpha() and not (len(symbol) > 1 and symbol[1].islower())):
            pseudopotentials[symbol] = pp   
        
k_points = 8
scf_params = {
    'calculation': 'scf',
    'tprnfor': True,
    'tstress': True,
    'input_data': {
        'system': {
            'ecutwfc': 30 * units.eV,  # PW cutoff
            'ecutrho': 240 * units.eV,  # Charge cutoff
            'ibrav': 0
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_beta': 0.5,
            'conv_thr': 1e8,
        }
    },
    'pseudopotentials': pseudopotentials,
    'kpts': (k_points, k_points, k_points),  # k-points grid, adjust as needed
    'parallel': 'all',
    'directory': outdir,  # Custom directory for calculation files
    'label': reduced_formula,  # Prefix for the filenames
    'logfile': "{}.log".format(reduced_formula),  # Logfile name
    'command': "mpirun -np 8 " + PWSCF_COMMAND + " -in {}.pwi > {}.pwo".format(reduced_formula, reduced_formula),
    'pseudo_dir': PPDIR,
}

bands_params = deepcopy(scf_params)
bands_params['calculation'] = 'bands'
bands_params['restart_mode'] = 'restart'
bands_params['verbosity'] = 'high'
bands_params['label'] = reduced_formula + '_bands'
bands_params['logfile'] = reduced_formula + '_bands.log'
bands_params['command'] = "mpirun -np 8 " + PWSCF_COMMAND + " -in {}.pwi > {}.pwo".format(reduced_formula + '_bands', reduced_formula + '_bands')

In [ ]:
# Attach the calculator to the structures
calculator = Espresso(**scf_params)
lattice.set_calculator(calculator)
energy = lattice.get_potential_energy()
fermi = calculator.get_fermi_level()

final_structure = read('{}/{}.pwo'.format(outdir, reduced_formula)) 

In [ ]:
band_calculator = Espresso(**bands_params)
final_structure.set_calculator(band_calculator)
band_calculator.calculate(final_structure)